In [1]:
!pip install numpy
!pip install torch
!pip install -U scikit-learn
!pip install panda


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from giza_datasets import DatasetsHub, DatasetsLoader
from sklearn.model_selection import KFold

# Define the number of folds
num_folds = 5

# Initialize k-fold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True)

# Load the dataset using DatasetsLoader
loader = DatasetsLoader()
df_polar = loader.load('yearn-individual-deposits')
df = df_polar.to_pandas()

# Define neural network parameters
input_size = 6
hidden_size1 = 64
hidden_size2 = 32
output_size = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 50

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Prepare features and target
features = df[['evt_block_number', 'token_decimals', 'evt_block_time']]
target = df['value']

# Convert 'evt_block_time' to datetime
features['evt_block_time'] = pd.to_datetime(features['evt_block_time'])

# Extract time-based features
features['hour_of_day'] = features['evt_block_time'].dt.hour
features['day_of_week'] = features['evt_block_time'].dt.dayofweek
features['month'] = features['evt_block_time'].dt.month

# Drop 'evt_block_time'
features = features.drop(columns=['evt_block_time'])

# Encode 'token_contract_address' as categorical
unique_addresses = sorted(set(df['token_contract_address']))
address_to_label = {addr: i for i, addr in enumerate(unique_addresses)}
encoded_addresses = [address_to_label[addr] for addr in df['token_contract_address']]
features['token_contract_address'] = encoded_addresses

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train multiple instances of the neural network model
models = []
for _ in range(5):  # Number of models in the ensemble
    model = NeuralNet(input_size, hidden_size1, hidden_size2, output_size)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i in range(0, len(x_train), batch_size):
            inputs = torch.tensor(x_train.iloc[i:i+batch_size].values).to(device)
            labels = torch.tensor(y_train.iloc[i:i+batch_size].values).to(device)
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs.flatten(), labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')
    models.append(model)

# Aggregate predictions from each model
ensemble_predictions = np.zeros(len(x_test))
for model in models:
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(x_test.values).to(device)
        outputs = model(inputs.float()).cpu().numpy().flatten()
        ensemble_predictions += outputs

ensemble_predictions /= len(models)

# Evaluate the ensemble
ensemble_rmse = np.sqrt(mean_squared_error(y_test, ensemble_predictions))
print(f'Ensemble RMSE: {ensemble_rmse}')


C:\Users\CCL\AppData\Local\Temp\ipykernel_22452\2400028415.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['evt_block_time'] = pd.to_datetime(features['evt_block_time'])
C:\Users\CCL\AppData\Local\Temp\ipykernel_22452\2400028415.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['hour_of_day'] = features['evt_block_time'].dt.hour
C:\Users\CCL\AppData\Local\Temp\ipykernel_22452\2400028415.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

Epoch [10/50], Loss: 1303153201049600.0000
Epoch [20/50], Loss: 1303118644716288.0000
Epoch [30/50], Loss: 1303104787204864.0000
Epoch [40/50], Loss: 1303112166520064.0000
Epoch [50/50], Loss: 1303119832615552.0000
Epoch [10/50], Loss: 1303159870256896.0000
Epoch [20/50], Loss: 1303126632371712.0000
Epoch [30/50], Loss: 1303159195200768.0000
Epoch [40/50], Loss: 1303108128784896.0000
Epoch [50/50], Loss: 1303102814770944.0000
Epoch [10/50], Loss: 1303337521751680.0000
Epoch [20/50], Loss: 1303131024423936.0000
Epoch [30/50], Loss: 1303114061883392.0000
Epoch [40/50], Loss: 1303106686244352.0000
Epoch [50/50], Loss: 1303099528033536.0000
Epoch [10/50], Loss: 1303182713294080.0000
Epoch [20/50], Loss: 1303229939470336.0000
Epoch [30/50], Loss: 1303124854963456.0000
Epoch [40/50], Loss: 1303111658442752.0000
Epoch [50/50], Loss: 1303103888573440.0000
Epoch [10/50], Loss: 1303347298149632.0000
Epoch [20/50], Loss: 1303210873505792.0000
Epoch [30/50], Loss: 1303112672585472.0000
Epoch [40/5